In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dir = "/content/drive/My Drive/bttai-ajl-2025/train/train"
test_dir = "/content/drive/My Drive/bttai-ajl-2025/test/test"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (299, 299)  # For InceptionV3
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

Found 2300 images belonging to 21 classes.
Found 560 images belonging to 21 classes.


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image

test_images = sorted(os.listdir(test_dir))  # Sort for consistency
test_data = []

for img_name in test_images:
    img_path = os.path.join(test_dir, img_name)
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    test_data.append(img_array)

test_data = np.array(test_data)


In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load Pretrained Model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Add Custom Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(len(train_generator.class_indices), activation='softmax')(x)  # Output classes

model = Model(inputs=base_model.input, outputs=x)

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(train_generator, validation_data=val_generator, epochs=5)

Epoch 1/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 675s 9s/step - accuracy: 0.2082 - loss: 2.7664 - val_accuracy: 0.3607 - val_loss: 2.1315
Epoch 2/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 639s 9s/step - accuracy: 0.4322 - loss: 1.8102 - val_accuracy: 0.3821 - val_loss: 2.0250
Epoch 3/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 654s 9s/step - accuracy: 0.5504 - loss: 1.4583 - val_accuracy: 0.4018 - val_loss: 1.9084
Epoch 4/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 653s 9s/step - accuracy: 0.6310 - loss: 1.1866 - val_accuracy: 0.4464 - val_loss: 1.8788
Epoch 5/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 632s 9s/step - accuracy: 0.6936 - loss: 0.9960 - val_accuracy: 0.4304 - val_loss: 1.9060


In [ ]:
predictions = model.predict(test_data)
predicted_labels = [list(train_generator.class_indices.keys())[i] for i in predictions.argmax(axis=1)]

39/39 ━━━━━━━━━━━━━━━━━━━━ 269s 7s/step


In [ ]:
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc:.4f}")


18/18 ━━━━━━━━━━━━━━━━━━━━ 125s 7s/step - accuracy: 0.4406 - loss: 1.8701
Validation Accuracy: 0.4304


In [ ]:
import pandas as pd

submission_df = pd.DataFrame({
    "md5hash": [name[:-4] for name in test_images],  # Remove ".jpg"
    "label": predicted_labels
})

submission_df.to_csv("submission.csv", index=False)